<a href="https://colab.research.google.com/github/parthag1201/FirstCrew/blob/main/CalciCrew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Installing the dependencies
!pip install crewai langchain-community pydantic
!pip install 'crewai[tools]'
!pip install 'crewai[CrewStructuredTool]'
!pip install -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.1 MB/s eta 0:00:

In [3]:
# Using required packages

from crewai import Crew, Task, Agent, Process, LLM
from pydantic import BaseModel
from crewai.tools.structured_tool import CrewStructuredTool
from crewai.tools import tool
from langchain_community.tools import BraveSearch
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.llms import Ollama  # If we use Ollama instead of Gemini
from google.colab import userdata # For API Secret

In [26]:
# Choose LLM

llm = LLM(
    model='gemini/gemini-2.0-flash',  # Use 'vertex_ai' as the provider for Google's Vertex AI
    api_key=userdata.get('API_KEY_1')
)

In [27]:
# Define Web Search Engine

@tool("duckduckgo_search_tool")
def duckduckgo_search_wrapper(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    search_tool = DuckDuckGoSearchRun(region="pt-br")
    result = search_tool.run(query)
    return result

In [28]:
product_manager_agent = Agent(
    role="Product Visionary",
    goal=(
        "To define the product vision, strategy, and roadmap for the {topic}. Oversee market research, competitor analysis, "
        "and feature prioritization to ensure alignment with business objectives."
    ),
    backstory=(
        "You are a seasoned product manager in software development with experience in leading cross-functional teams. Your expertise "
        "lies in crafting compelling product strategies that drive market success and deliver sde products."
    ),
    tool = ["duckduckgo_search_tool"],
    allow_delegation = 1,
    llm=llm,
    verbose=True
)

business_analyst_agent = Agent(
    role="Business Strategist",
    goal=(
        "To gather and analyze business requirements, translate them into technical specifications, on the {topic} and ensure "
        "that solutions meet market needs."
    ),
    backstory=(
        "You are a skilled business analyst with a deep understanding of market dynamics and technical "
        "requirements. Your expertise helps bridge the gap between business stakeholders and developers."
    ),
    llm=llm,
    allow_delegation = 1,
    verbose=True
)

software_architect_agent = Agent(
    role="Technical Architect",
    goal=(
        "To design the overall structure of the {topic} software system, ensuring scalability, performance, and security. "
        "Provide technical guidance and oversight throughout the development lifecycle."
    ),
    backstory=(
        "A seasoned software architect with expertise in designing robust and scalable software systems. "
        "You lead technical teams in implementing best practices and ensuring high-quality code."
        "You are skilled in frontend and backend development of software products"
    ),
    llm=llm,
    allow_delegation = 1,
    verbose=True
)

development_team_lead_agent = Agent(
    role="Technical Lead",
    goal=(
        "To guide the development team in translating requirements into high-quality code. Ensure code quality, "
        "provide mentorship, and solve technical challenges."
        "Code the {topic}"
    ),
    backstory=(
        "You are an experienced technical lead with a strong background in software development. Your role is "
        "to mentor developers and ensure that all code aligns with project guidelines."
    ),
    llm=llm,
    allow_delegation = 1,
    verbose=True
)

ux_ui_designer_agent = Agent(
    role="User Experience Designer",
    goal=(
        "To craft user interfaces for {topic} that are aesthetically pleasing and user-friendly. Ensure that the design aligns "
        "with the product's vision and user needs."
    ),
    backstory=(
        "A skilled UX/UI designer with expertise in creating intuitive and engaging user experiences. Your work "
        "focuses on making software applications both visually appealing and functional."
    ),
    llm=llm,
    allow_delegation = 1,
    verbose=True
)

manager_agent = Agent(
    role="Workflow Maestro",
    goal=(
        "To coordinate agents, manage task dependencies, and ensure all outputs meet quality standards. Your focus "
        "is on delivering a cohesive final product through efficient task management."
        "You can interact with various agents to co-ordinate the product requirements, techstack breakdwon and drawing a technical architecture and overseeing its implementation."
        "The solution you deliver must a cutting state-of-art technological product that showcases the modern techniques"
    ),
    backstory=(
        "A former project manager with a passion for efficient teamwork, you ensure every process runs smoothly, "
        "overseeing tasks and verifying results."
    ),
    llm=llm,
    allow_delegation = 1,
    verbose=True
)

In [29]:
product_management_task = Task(
    description=(
        "Oversee the entire project lifecycle, manage timelines, allocate resources, and ensure projects are "
        "completed on time and within budget for the software product: {topic}."
    ),
    expected_output=(
        "Regular project status updates and a final project report detailing progress, challenges, and outcomes."
    ),

    agent=product_manager_agent
)

market_requirements_task = Task(
    description=(
        "Gather and document business and technical requirements for the software product : {topic}. Ensure alignment with "
        "market research findings."
    ),
    expected_output=(
        "A detailed document outlining product requirements, including user stories and technical specifications on the software product : {topic}."
    ),
    agent=business_analyst_agent
)

software_architecture_task = Task(
    description=(
        "Design the overall architecture of the software system, ensuring scalability, performance, and security on the software product : {topic}."
    ),
    expected_output=(
        "A detailed architecture document outlining system components, interactions, and technical specifications on the software product : {topic}."
    ),
    agent = software_architect_agent
)

development_task = Task(
    description=(
        "Implement the software product based on the defined architecture and requirements. Ensure high-quality "
        "code and adherence to coding standards."
    ),
    expected_output=(
        "A fully functional software product with well-documented code and tests."
    ),
    agent = development_team_lead_agent
)

ux_ui_design_task = Task(
    description=(
        "Create user interfaces that are intuitive and visually appealing on the software product : {topic}. Ensure design aligns with product vision "
        "and user needs."
    ),
    expected_output=(
        "Design prototypes and specifications for user interfaces on the software product : {topic}."
    ),

    agent = ux_ui_designer_agent
)

workflow_coordination_task = Task(
    description=(
        "Coordinate agents to deliver the required software product"
        "Product Requirement : {topic}. "
        "Manage task dependencies and ensure all outputs meet quality standards."
    ),
    expected_output=(
        "Seemless co-ordination between all the agents simulating how software prodcuts are built in MNCs."
    ),

    agent = manager_agent
)

In [30]:
crew = Crew(
    agents=[
        product_manager_agent, business_analyst_agent, software_architect_agent,
        development_team_lead_agent, ux_ui_designer_agent,
    ],
    tasks=[
        product_management_task, market_requirements_task, software_architecture_task, development_task,
        ux_ui_design_task
    ],
    process=Process.hierarchical,
    manager_agent=manager_agent,
    verbose=True
)


In [31]:
# Start Crew.ai by giving the required input

crew_output = crew.kickoff(inputs={"topic": "Creating a advanced Calculator Webpage using HTML, CSS, and JavaScript. "})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 815a9b36-f6a7-4579-bf14-fa95eb79eb32                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f6693db3-b9f1-4331-acdc-12f6108282f7
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f6693db3-b9f1-4331-acdc-12f6108282f7
       Status: Executing Task...
    └── 🤖 Agent: Workflow Maestro
            Status: In Progress

# Agent: Workflow Maestro
## Task: Oversee the entire project lifecycle, manage timelines, allocate resources, and ensure projects are completed on time and within budget for the software product: Creating a advanced Calculator Webpage using HTML, CSS, and JavaScript. .


🚀 Crew: crew
└── 📋 Task: f6693db3-b9f1-4331-acdc-12f6108282f7
       Status: Executing Task...
    └── 🤖 Agent: Workflow Maestro
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Workflow Maestro
## Final Answer: 
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Advanced Calculator</title>
    <link rel="stylesheet" href="style.css">
    <style>
        /* Updated CSS for improved layout and theming */
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f4;
            display: flex;
            flex-direction: column;
            min-height: 100vh;
            transition: background-color 0.3s, color 0.3s; /* Smooth transitions */
        }

        header {
            background-color: #333;
            color: #fff;
            padding: 1em 0;
            text-align: center;
            transition: background-color 0.3s, color 0.3s;
        }

        footer {
            background-color: #333;
            color: #fff;
            text-align: center

🚀 Crew: crew
├── 📋 Task: f6693db3-b9f1-4331-acdc-12f6108282f7
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (5)
│   ├── 🤖 Agent: Product Visionary
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (25)
│   ├── 🤖 Agent: Product Visionary
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (26)
│   ├── 🤖 Agent: Product Visionary
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (27)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 14808651-a511-4850-87b4-52e0606e8dd9
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (29)
│   ├── 🤖 Agent: Business Strategist
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (30)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: a8bd9a33-6e9d-4a8c-adcf-48f53c984d15
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (32)
│   ├── 🤖 Agent: Technical Architect
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (34)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: c1c25be6-dbf6-4b78-84c2-4a7f4af001d8
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (36)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
└── 📋 Task: a131c150-4584-431d-a3e9-3cdbd204f20e
       Status: Executing Task...
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (38)
    └── 🤖 Agent: Workflow Maestro
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: f6693db3-b9f1-4331-acdc-12f6108282f7
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (5)
│   ├── 🤖 Agent: Product Visionary
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (25)
│   ├── 🤖 Agent: Product Visionary
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (26)
│   ├── 🤖 Agent: Product Visionary
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (27)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 14808651-a511-4850-87b4-52e0606e8dd9
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (29)
│   ├── 🤖 Agent: Business Strategist
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (30)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: a8bd9a33-6e9d-4a8c-adcf-48f53c984d15
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (32)
│   ├── 🤖 Agent: Technical Architect
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (34)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: c1c25be6-dbf6-4b78-84c2-4a7f4af001d8
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (36)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
└── 📋 Task: a131c150-4584-431d-a3e9-3cdbd204f20e
       Assigned to: Workflow Maestro
       Status: ✅ Completed
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (38)
    └── 🤖 Agent: Workflow Maestro
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a131c150-4584-431d-a3e9-3cdbd204f20e                                                                     │
│  Agent: Workflow Maestro                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 815a9b36-f6a7-4579-bf14-fa95eb79eb32                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [32]:
# Output

print(crew_output)

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Advanced Calculator</title>
    <link rel="stylesheet" href="style.css">
    <style>
        /* Updated CSS for improved layout and theming */
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f4;
            display: flex;
            flex-direction: column;
            min-height: 100vh;
            transition: background-color 0.3s, color 0.3s; /* Smooth transitions */
        }

        header {
            background-color: #333;
            color: #fff;
            padding: 1em 0;
            text-align: center;
            transition: background-color 0.3s, color 0.3s;
        }

        footer {
            background-color: #333;
            color: #fff;
            text-align: center;
            padding: 1em 0;
            marg